In [ ]:
import requests
from urllib.parse import quote
import urllib
import json
from pandas import json_normalize
import pandas as pd
import numpy as np
import datetime as dt 

In [ ]:
f = open('인증키 파일경로 또는 이름', 'r')
ServiceKey = f.read()
len(ServiceKey)

In [ ]:
years = [2016, 2017, 2018, 2019, 2020, 2021]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
operation = ['getRestDeInfo','getAnniversaryInfo']

In [ ]:
# 공휴일
r_day = []
hol_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'

for year in years:
    for month in months:
        option = f'solYear={year}&solMonth={month}&ServiceKey={ServiceKey}&_type=json'
        url = f'{hol_url}?{option}'
        result = requests.get(url).json()
        r_day.append(result)
#r_day[0]['response']['body']['items']['item']
r_day

In [ ]:
# 기념일
ann_day = []
hol_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo'

for year in years:
    for month in months:
        option = f'solYear={year}&solMonth={month}&ServiceKey={ServiceKey}&_type=json'
        url = f'{hol_url}?{option}'
        result = requests.get(url).json()
        ann_day.append(result)
#ann_day[0]['response']['body']['items']['item']
ann_day

In [ ]:
# 공휴일 데이터프레임 만들기
rday_df = pd.DataFrame.from_dict(json_normalize(r_day), orient='columns')
rday_df

In [ ]:
# 기념일 데이터프레임 만들기
annday_df = pd.DataFrame.from_dict(json_normalize(ann_day), orient='columns')
annday_df

In [ ]:
# 공휴일 - 필요한 열만 추출
rdf = rday_df[['response.body.items.item.dateName','response.body.items.item.isHoliday','response.body.items.item.locdate','response.body.items.item']]
rdf

In [ ]:
# 기념일 - 필요한 열만 추출
anndf = annday_df[['response.body.items.item.dateName', 'response.body.items.item.isHoliday','response.body.items.item.locdate','response.body.items.item']]
anndf['response.body.items.item'][4]

In [ ]:
# column명 변경 
rdf.columns = ['name','isholiday','date', 'name2']
anndf.columns = ['name','desc','isholiday','date']

In [ ]:
print('--------------------공휴일-------------------------')
print(rdf)
print('--------------------기념일-------------------------')
print(anndf)

In [ ]:
# 데이터 확인 
rdf

In [ ]:
# name 기준 df 만들기 
rdf1 = rdf.dropna(subset=['name'])
rdf1 = rdf1[['name', 'date']]
rdf1

In [ ]:
# name2 기준 dateName, locdate 추출 
rdf2_lis=[]
for item in rdf['name2']:
    if type(item)==list:
        for i in range(len(item)):
            r =[]
            r.append(item[i]['locdate'])
            r.append(item[i]['dateName'])
            if r != []:
                rdf2_lis.append(r)       
rdf2_lis

In [ ]:
# 추출한 데이터 데이터프레임 만들기 
rrdf2 = pd.DataFrame(rdf2_lis, columns=['date', 'name'])
rdf2

In [ ]:
# column 순서 바꾸기 
rdf1 = rdf1.reindex(columns=['date', 'name'])
rdf1

In [ ]:
# 데이터 갯수 확인
rdf1.shape, rdf2.shape

In [ ]:
# name, name2 기준으로 된 rdf1, rdf2 병합 
rdf_new = pd.merge(rdf1, rdf2, how='outer')
rdf_new

In [ ]:
# 데이터 타입 확인 
rdf_new.info()

In [ ]:
# 데이터 타입 변환 (문자열 변환 후 슬라이싱까지)
rdf_new['date'] = rdf_new['date'].astype('str').str[:8]
rdf_new

In [ ]:
# 시간으로 바꿔주기 
import datetime as dt
rdf_new['date'] = pd.to_datetime(rdf_new['date'])
rdf_new

In [ ]:
# 년,월,일, 요일 구분 후 순서조정(시계열)
rdf_new['year'] = rdf_new['date'].dt.year
rdf_new['month'] = rdf_new['date'].dt.month
rdf_new['day'] = rdf_new['date'].dt.day
rdf_new['wday'] = rdf_new['date'].dt.weekday    #월요일0 - 일요일6

rdf_new[['date', 'year','month', 'day', 'wday', 'name']]

In [ ]:
# 날짜순으로 정리 후 인덱스 수정 
rdf_new = rdf_new.sort_values('date')
rdf_new.reset_index(drop=True, inplace=True)

- 기념일

In [ ]:
## 기념일 
anndf

In [ ]:
anndf['date'][4]

In [ ]:
# name에 들어있는 date 추출(thx to 나영님)
anndf1_lis = []
for i in range(len(anndf)):
    if pd.notna(anndf['name'][i]):
        anndf1_lis.append([anndf['isholiday'][i], anndf['name'][i]])

anndf1_lis

In [ ]:
# date에 들어있는 date 추출(thx to 태혁님)
anndf2_lis=[]
for item in anndf['date']:
    if type(item)==list:
        for i in range(len(item)):
            r =[]
            r.append(item[i]['locdate'])
            r.append(item[i]['dateName'])
            # r.append(item[i]['isHoliday'])
            if r != []:
                anndf2_lis.append(r)       
anndf2_lis

In [ ]:
# 데이터프레임만들기
anndf1 = pd.DataFrame(anndf1_lis, columns=['date', 'name'])
anndf2 = pd.DataFrame(anndf2_lis, columns=['date', 'name'])

In [ ]:
# 병합
anndf_new = pd.merge(anndf1, anndf2, how='outer')
anndf_new

In [ ]:
# 문자열로 바꿔서 잘라주고
anndf_new['date'] = anndf_new['date'].astype('str').str[:8]

# 날짜타입으로 변경
anndf_new['date'] = pd.to_datetime(anndf_new['date'])
anndf_new['year'] = anndf_new['date'].dt.year
anndf_new['month'] = anndf_new['date'].dt.month
anndf_new['day'] = anndf_new['date'].dt.day
anndf_new['wday'] = anndf_new['date'].dt.weekday        #월요일0 - 일요일6

anndf_new[['date', 'year','month', 'day', 'wday', 'name']]

In [ ]:
# 인덱스를 name으로 설정 
anndf_new = anndf_new.set_index('name')
anndf_new

In [ ]:
# 그나마 의미있는 것은 '근로자의날' 이라서 그것만 추출
lbday = anndf_new.loc['근로자의 날']
lbday

In [ ]:
# 방법1 - 파이썬 컴프리헨션(압축기법)
df['sugar']=[x.split('/')[0] if pd.notnull(x) else x for x in tmp['sugar_PH']]
df['PH']=[x.split('/')[1] if pd.notnull(x) else x for x in tmp['sugar_PH']]

# df['sugar'] = df['sugar_PH'].apply(
#     lambda x: x.split('/')[0] if pd.notnull(x) else x)

# df['PH'] = df['sugar_PH'].apply(
#     lambda x: x.split('/')[1] if pd.notnull(x) else x)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 6496
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     6497 non-null   int64  
 1   alcohol   6497 non-null   float64
 2   class     6493 non-null   float64
 3   sugar_PH  6497 non-null   object 
 4   sugar     6497 non-null   object 
 5   PH        6497 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 355.3+ KB


In [ ]:
# 인덱스 다시 리셋하고 열추가(rdf_new랑 merge할거니까)
# drop: 인덱스세팅한 열 삭제(True) or 유지(False)
# inplace: 원본데이터반환(True) or 복사본 반환(False)
lbday = lbday.reset_index(drop=True, inplace=False) # index(0~)열 삭제 
lbday['name']='근로자의 날'
lbday

In [ ]:
hdf = pd.merge(rdf_new, lbday, how='outer')
hdf

In [ ]:
hdf.info()

In [ ]:
## 날짜(date)순으로 정렬하고 인덱스 리셋
# ascending=False 하면 내림차순
# df = df.sort_values(by=['칼럼1'], axis=0, ascending = False)
# axis = 0 하면 세로방향으로 있는 데이터 정렬(우리가 흔히 쓰는거) 따라서 by = '칼럼이름'
# axis = 1 하면 가로방향으로 있는 데이터 정렬 따라서 by = 'row번호'
hdf = hdf.sort_values('date')
hdf.reset_index(drop=True, inplace=True)
hdf

In [ ]:
hdf.to_csv('nationalholiday.csv')